# Overview
The notebook shows how you can interact with XNAT and see the projects, subjects, experiments and even extract some of the data

In [1]:
from __future__ import print_function, division, absolute_import
import pip
try:
    import pyxnat
except ImportError as ie:
    print('pyxnat not installed {}, lets try and install it'.format(ie))
    pip.main(['install','pyxnat'])
try:
    import pydicom
except ImportError as ie:
    print('pydicom >= 1.0 not installed {}, lets try and install it'.format(ie))
    pip.main(['install', 'git+https://github.com/pydicom/pydicom'])

In [2]:
from pyxnat import Interface

In [3]:
interface = Interface(
              server='http://0.0.0.0/xnat',
              user='admin',
              password='admin'
              )

In [4]:
all_projects = interface.select.projects().get()
print('# of projects', len(all_projects), 'taking first project', all_projects[0])

# of projects 1 taking first project lowdose


In [5]:
all_subjects = interface.select('/projects/*/subjects').get()
print('# of subjects', len(all_subjects), 'taking first project', all_subjects[0])

# of subjects 1 taking first project XNAT_S00001


In [6]:
cur_project = all_projects[0]
cur_proj_ref = interface.select.project(cur_project)

# Traverse the Tree
The data are hierarchircally organized by subject, experiment, scan, resource, file and so to get to the actual data we have to go through each step. In the dummy case there is only one, but typically there are many more

In [7]:
all_subjects = cur_proj_ref.subjects().get()
print('Project Subjects:', all_subjects)
cur_subj_ref = cur_proj_ref.subject(all_subjects[0])

all_experiments = cur_subj_ref.experiments().get()
print('Subject Experiments:', all_experiments)
cur_exp_ref = cur_subj_ref.experiment(all_experiments[0])

all_scans = cur_exp_ref.scans().get()
print('Scans', all_scans)
cur_scan_ref = cur_exp_ref.scan(all_scans[0])

all_resources = cur_scan_ref.resources().get()
print('Resources', all_resources)
cur_res_ref = cur_scan_ref.resource(all_resources[0])

Project Subjects: ['XNAT_S00001']
Subject Experiments: ['XNAT_E00001']
Scans ['3']
Resources ['1']


In [8]:
all_files = cur_res_ref.files().get()
print('Files #', len(all_files))
cur_file_ref = cur_res_ref.file(all_files[0])
print('Format', cur_file_ref.format(), 'Size', cur_file_ref.size())
print('Attributes', cur_file_ref.attributes())

Files # 395
Format DICOM Size 528336
Attributes {'Name': '1.2.840.113619.2.40.124042.4.2147483647.1502172013.117467', 'file_format': 'DICOM', 'file_content': '', 'URI': '/data/projects/lowdose/subjects/XNAT_S00001/experiments/XNAT_E00001/scans/3/resources/1/files/1.2.840.113619.2.40.124042.4.2147483647.1502172013.117467', 'file_tags': '', 'path': '1.2.840.113619.2.40.124042.4.2147483647.1502172013.117467', 'Size': '528336'}


# Downloading Data
It appears the data is not yet downloaded in a usuable form and for some reason we get JSON files instead of DICOM files. 
- Reference: https://pythonhosted.org/pyxnat/features/files.html does not seem to clarify much

In [9]:
from pydicom import read_file
cur_file_ref.get_copy('./test2.dcm')
try:
    in_image = read_file('test2.dcm')
    print(in_image)
except Exception as e:
    print('Could not read as dicom {}'.format(e))
    import json
    with open('test2.dcm') as f:
        c_data = json.load(f)
    print(c_data)

Could not read as dicom File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
{u'ResultSet': {u'Result': [{u'Name': u'1.2.840.113619.2.40.124042.4.2147483647.1502172013.117467', u'file_format': u'DICOM', u'file_content': u'', u'cat_ID': u'1', u'URI': u'/data/projects/lowdose/subjects/XNAT_S00001/experiments/XNAT_E00001/scans/3/resources/1/files/1.2.840.113619.2.40.124042.4.2147483647.1502172013.117467', u'collection': u'DICOM', u'file_tags': u'', u'digest': u'040bdbaad2a300efd18474d807639a91', u'Size': u'528336'}, {u'Name': u'1.2.840.113619.2.40.124042.4.2147483647.1502172015.870530', u'file_format': u'DICOM', u'file_content': u'', u'cat_ID': u'1', u'URI': u'/data/projects/lowdose/subjects/XNAT_S00001/experiments/XNAT_E00001/scans/3/resources/1/files/1.2.840.113619.2.40.124042.4.2147483647.1502172015.870530', u'collection': u'DICOM', u'file_tags': u'', u'digest': u'beaa377a48ac27d9bb5bfe1f09a175ef', u'Size':